In [124]:
#dependencies
import pandas as pd
from bs4 import BeautifulSoup
import pymongo
from splinter import Browser
import requests
from config import api_key
import json

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [70]:
url='https://www.the-numbers.com/movie/budgets/all'
browser.visit(url)

In [71]:
movie_df=[]

In [72]:
#scrapes top 500 movie production budgets from the website the-numbers
page_list=['101-','201-','301-','401-','501-']
for page in page_list:
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    table=soup.find_all('table')[0]
    df=pd.read_html(str(table))
    df=df[0]
    df=df.drop(columns='Unnamed: 0')
    movie_df.append(df)
    browser.click_link_by_partial_text(page)

In [111]:
#appends movie data into one dataframe 
movies_df=movie_df[0].append(movie_df[1]).append(movie_df[2]).append(movie_df[3]).append(movie_df[4]).reset_index().drop(columns='index')

In [112]:
movies_df

,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross
0,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,797,800,564"
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,063,875","$1,045,663,875"
2,"Apr 22, 2015",Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,403,013,963"
3,"Dec 16, 2015",Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,068,223,624"
4,"Apr 25, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,359,754"
...,...,...,...,...,...
495,"Jul 7, 2004",King Arthur,"$90,000,000","$51,877,963","$203,877,963"
496,"Apr 25, 1997",Volcano,"$90,000,000","$47,546,796","$120,100,000"
497,"Mar 26, 1997",The Devil's Own,"$90,000,000","$42,885,593","$140,900,000"
498,"Jul 19, 2002",K-19: The Widowmaker,"$90,000,000","$35,168,966","$65,716,126"


In [113]:
#OMDB url
url=f'http://www.omdbapi.com/?t='
#empty lists to be appended later
imdb_rating_list=[]
metascore_list=[]
rotten_tomatoes_score_list=[]
rated_list=[]
#OMDB api call for ratings
for movie in movies_df['Movie']:
    response=requests.get(url+movie+'&apikey='+api_key)
    data=response.json()
    try:
        
        try:
            imdb_rating=data['Ratings'][0]['Value']
        except:
            imdb_rating='NaN'
        metascore=data['Metascore']
        try:
            rotten_tomatoes=data['Ratings'][1]['Value']
        except IndexError:
            rotten_tomatoes='NaN'
        rated=data['Rated']
    except:
        imdb_rating='NaN'
        rotten_tomatoes='NaN'
        metascore='NaN'
        rated='NaN'
    metascore_list.append(metascore)
    imdb_rating_list.append(imdb_rating)
    rotten_tomatoes_score_list.append(rotten_tomatoes)
    rated_list.append(rated)
    

In [114]:
#adding lists to movies DataFrame
movies_df['Rated']=rated_list
movies_df['IMDB_Rating']=imdb_rating_list
movies_df['Metascore']=metascore_list
movies_df['Rotten Tomatoes']=rotten_tomatoes_score_list
movies_df.head()

,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross,Rated,IMDB_Rating,Metascore,Rotten Tomatoes
0,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,797,800,564",PG-13,8.5/10,78,94%
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,063,875","$1,045,663,875",PG-13,6.6/10,45,33%
2,"Apr 22, 2015",Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,403,013,963",PG-13,7.3/10,66,75%
3,"Dec 16, 2015",Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,068,223,624",NaN,NaN,NaN,NaN
4,"Apr 25, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,359,754",PG-13,8.5/10,68,85%


In [115]:
#droping all rows with NaN values
movies_NaN=movies_df[movies_df['Rotten Tomatoes']=='NaN'].index
movies_df=movies_df.drop(movies_NaN)
movies_df=movies_df.reset_index()
movies_df=movies_df.drop(columns=['index'])
movies_df


,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross,Rated,IMDB_Rating,Metascore,Rotten Tomatoes
0,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,797,800,564",PG-13,8.5/10,78,94%
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,063,875","$1,045,663,875",PG-13,6.6/10,45,33%
2,"Apr 22, 2015",Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,403,013,963",PG-13,7.3/10,66,75%
3,"Apr 25, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,359,754",PG-13,8.5/10,68,85%
4,"May 24, 2007",Pirates of the Caribbean: At World’s End,"$300,000,000","$309,420,425","$963,420,425",PG-13,7.1/10,50,44%
...,...,...,...,...,...,...,...,...,...
459,"Jul 7, 2004",King Arthur,"$90,000,000","$51,877,963","$203,877,963",PG-13,6.3/10,46,31%
460,"Apr 25, 1997",Volcano,"$90,000,000","$47,546,796","$120,100,000",PG-13,5.5/10,55,50%
461,"Mar 26, 1997",The Devil's Own,"$90,000,000","$42,885,593","$140,900,000",R,6.2/10,53,34%
462,"Jul 19, 2002",K-19: The Widowmaker,"$90,000,000","$35,168,966","$65,716,126",PG-13,6.7/10,58,61%


In [116]:
#drops values in Metascore with N/A
movies_na=movies_df[movies_df['Metascore']=='N/A'].index
movies_df=movies_df.drop(movies_na)
movies_df=movies_df.reset_index()
movies_df=movies_df.drop(columns=['index'])
movies_df

,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross,Rated,IMDB_Rating,Metascore,Rotten Tomatoes
0,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,797,800,564",PG-13,8.5/10,78,94%
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,063,875","$1,045,663,875",PG-13,6.6/10,45,33%
2,"Apr 22, 2015",Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,403,013,963",PG-13,7.3/10,66,75%
3,"Apr 25, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,359,754",PG-13,8.5/10,68,85%
4,"May 24, 2007",Pirates of the Caribbean: At World’s End,"$300,000,000","$309,420,425","$963,420,425",PG-13,7.1/10,50,44%
...,...,...,...,...,...,...,...,...,...
458,"Jul 7, 2004",King Arthur,"$90,000,000","$51,877,963","$203,877,963",PG-13,6.3/10,46,31%
459,"Apr 25, 1997",Volcano,"$90,000,000","$47,546,796","$120,100,000",PG-13,5.5/10,55,50%
460,"Mar 26, 1997",The Devil's Own,"$90,000,000","$42,885,593","$140,900,000",R,6.2/10,53,34%
461,"Jul 19, 2002",K-19: The Widowmaker,"$90,000,000","$35,168,966","$65,716,126",PG-13,6.7/10,58,61%


In [117]:
#convert budget and gross income into numeric values from string
movies_df['ProductionBudget']=movies_df['ProductionBudget'].str.replace('$','').str.replace(',','')
movies_df['ProductionBudget']=movies_df['ProductionBudget'].astype(float)
movies_df['DomesticGross']=movies_df['DomesticGross'].str.replace('$','').str.replace(',','')
movies_df['DomesticGross']=movies_df['DomesticGross'].astype(float)
movies_df['WorldwideGross']=movies_df['WorldwideGross'].str.replace('$','').str.replace(',','')
movies_df['WorldwideGross']=movies_df['WorldwideGross'].astype(float)
movies_df

,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross,Rated,IMDB_Rating,Metascore,Rotten Tomatoes
0,"Apr 23, 2019",Avengers: Endgame,400000000.0,858373000.0,2.797801e+09,PG-13,8.5/10,78,94%
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,379000000.0,241063875.0,1.045664e+09,PG-13,6.6/10,45,33%
2,"Apr 22, 2015",Avengers: Age of Ultron,365000000.0,459005868.0,1.403014e+09,PG-13,7.3/10,66,75%
3,"Apr 25, 2018",Avengers: Infinity War,300000000.0,678815482.0,2.048360e+09,PG-13,8.5/10,68,85%
4,"May 24, 2007",Pirates of the Caribbean: At World’s End,300000000.0,309420425.0,9.634204e+08,PG-13,7.1/10,50,44%
...,...,...,...,...,...,...,...,...,...
458,"Jul 7, 2004",King Arthur,90000000.0,51877963.0,2.038780e+08,PG-13,6.3/10,46,31%
459,"Apr 25, 1997",Volcano,90000000.0,47546796.0,1.201000e+08,PG-13,5.5/10,55,50%
460,"Mar 26, 1997",The Devil's Own,90000000.0,42885593.0,1.409000e+08,R,6.2/10,53,34%
461,"Jul 19, 2002",K-19: The Widowmaker,90000000.0,35168966.0,6.571613e+07,PG-13,6.7/10,58,61%


In [118]:
#converts ratings from string to numeric values, IMDB_Raing change from being a score out of 10 to a score out of 100
movies_df['IMDB_Rating']=movies_df['IMDB_Rating'].str.replace('/10','')
movies_df['Rotten Tomatoes']=movies_df['Rotten Tomatoes'].str.replace('%','')
movies_df['Rotten Tomatoes']=movies_df['Rotten Tomatoes'].str.replace('/100','')
movies_df['IMDB_Rating']=movies_df['IMDB_Rating'].astype(float)
movies_df['Metascore']=movies_df['Metascore'].astype(float)
movies_df['Rotten Tomatoes']=movies_df['Rotten Tomatoes'].astype(float)
movies_df['IMDB_Rating']=movies_df['IMDB_Rating']*10

In [119]:
movies_df

,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross,Rated,IMDB_Rating,Metascore,Rotten Tomatoes
0,"Apr 23, 2019",Avengers: Endgame,400000000.0,858373000.0,2.797801e+09,PG-13,85.0,78.0,94.0
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,379000000.0,241063875.0,1.045664e+09,PG-13,66.0,45.0,33.0
2,"Apr 22, 2015",Avengers: Age of Ultron,365000000.0,459005868.0,1.403014e+09,PG-13,73.0,66.0,75.0
3,"Apr 25, 2018",Avengers: Infinity War,300000000.0,678815482.0,2.048360e+09,PG-13,85.0,68.0,85.0
4,"May 24, 2007",Pirates of the Caribbean: At World’s End,300000000.0,309420425.0,9.634204e+08,PG-13,71.0,50.0,44.0
...,...,...,...,...,...,...,...,...,...
458,"Jul 7, 2004",King Arthur,90000000.0,51877963.0,2.038780e+08,PG-13,63.0,46.0,31.0
459,"Apr 25, 1997",Volcano,90000000.0,47546796.0,1.201000e+08,PG-13,55.0,55.0,50.0
460,"Mar 26, 1997",The Devil's Own,90000000.0,42885593.0,1.409000e+08,R,62.0,53.0,34.0
461,"Jul 19, 2002",K-19: The Widowmaker,90000000.0,35168966.0,6.571613e+07,PG-13,67.0,58.0,61.0


In [120]:
#connecting to mongodb
conn='mongodb://localhost:27017'
client=pymongo.MongoClient(conn)

In [121]:
#creating new database
db=client.movies_db
db.command('dropDatabase')
collection=db.movies

In [122]:
#adding into movies_db
for i in movies_df.index:
    movie_dict=movies_df.iloc[i].to_dict()
    collection.insert_one(movie_dict)